## Stream attributes from Web events using Signals

This notebook creates a new view using the SDK, tests it on the atomic events table and applies.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["API_URL"],
    api_key=os.environ["API_KEY"],
    api_key_id=os.environ["API_KEY_ID"],
    org_id=os.environ["ORG_ID"],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
)

count_page_pings = Attribute(
    name="count_page_pings",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_ping",
            version="1-0-0",
        )
    ],
    aggregation="counter",
)

visited_pages = Attribute(
    name="visited_pages",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="unique_list",
    property="page_title",
)

### Wrapping the attribute in a view

All attributes need to be included in views that can be considered as "tables" of attributes.

Views are immutable and versioned.

In [ ]:
from snowplow_signals import View, session_entity

view = View(
    name="my_web_attributes",
    version=1,
    entity=session_entity,
    attributes=[
        count_page_pings,
        visited_pages,
    ],
)

### Testing the view

Execute the view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    view=view,
    app_ids=["website"],
)
data

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [ ]:
applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")